In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from pprint import pprint
import urllib
import time
import requests

In [2]:
NEWS_URL = "https://mars.nasa.gov/news/"
IMAGE_URL = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
WEATHER_URL = "https://twitter.com/marswxreport?lang=en"
FACTS_URL = "http://space-facts.com/mars/"
HEMISP_URL = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [3]:
driver = webdriver.Firefox()
driver.get(NEWS_URL)
html_news = driver.page_source

driver.get(IMAGE_URL)
html_image = driver.page_source

In [4]:
driver.get(WEATHER_URL)
time.sleep(3)
html_weather = driver.page_source

driver.get(FACTS_URL)
html_hemispheres = driver.page_source

driver.close()

# Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. 

Assign the text to variables that you can reference later.

In [5]:
# Grab Mars News
soup = BeautifulSoup(html_news, "html.parser")
news = soup.find("div", class_="list_text")

news_dict = {
    
    "Date": news.contents[0].text,
    "Title": news.contents[1].text,
    "Body": news.contents[2].text
}

pprint(news_dict)

{'Body': 'The pair of briefcase-sized satellites made history when they sailed '
         'past Mars in 2019.',
 'Date': 'February 27, 2020',
 'Title': 'The MarCO Mission Comes to an End'}


# Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use requests to get the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.


Example:
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'

In [6]:
# Grab latest JPL Mars Image
soup = BeautifulSoup(html_image, "html.parser")
mars_image = soup.find_all("a", class_="fancybox")

featured_image_url = urllib.parse.urljoin("https://www.jpl.nasa.gov", mars_image[1]["data-fancybox-href"])
pprint(featured_image_url)

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23707_hires.jpg'


# Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. 

Save the tweet text for the weather report as a variable called `mars_weather`.

In [7]:
soup = BeautifulSoup(html_weather, "html.parser")
mars_weather = soup.find("div", class_="css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
pprint(mars_weather.text)

('InSight sol 447 (2020-02-28) low -92.6ºC (-134.6ºF) high -11.1ºC (12.0ºF)\n'
 'winds from the SSW at 5.8 m/s (13.1 mph) gusting to 20.2 m/s (45.2 mph)\n'
 'pressure at 6.30 hPa')


# Visit the Mars Facts webpage [here](http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [8]:
request = requests.get(FACTS_URL)
mars_facts_soup = BeautifulSoup(request.text, "html.parser")

facts = f'{mars_facts_soup.find("table", class_="tablepress tablepress-id-p-mars")}'
pprint(facts)

('<table class="tablepress tablepress-id-p-mars" '
 'id="tablepress-p-mars"><tbody><tr class="row-1 odd"><td '
 'class="column-1"><strong>Equatorial Diameter:</strong></td><td '
 'class="column-2">6,792 km<br/></td></tr><tr class="row-2 even"><td '
 'class="column-1"><strong>Polar Diameter:</strong></td><td '
 'class="column-2">6,752 km<br/></td></tr><tr class="row-3 odd"><td '
 'class="column-1"><strong>Mass:</strong></td><td class="column-2">6.39 × '
 '10^23 kg<br/> (0.11 Earths)</td></tr><tr class="row-4 even"><td '
 'class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a '
 'href="https://space-facts.com/moons/phobos/">Phobos</a> &amp; <a '
 'href="https://space-facts.com/moons/deimos/">Deimos</a>)</td></tr><tr '
 'class="row-5 odd"><td class="column-1"><strong>Orbit '
 'Distance:</strong></td><td class="column-2">227,943,824 km<br/> (1.38 '
 'AU)</td></tr><tr class="row-6 even"><td class="column-1"><strong>Orbit '
 'Period:</strong></td><td class="column-2">687 da

# Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [9]:
driver = webdriver.Firefox()
driver.get(HEMISP_URL)
driver.implicitly_wait(10)


hem_list = []

links = driver.find_elements_by_class_name("thumb")

for l in range(len(links)):
    driver.find_elements_by_class_name("thumb")[l].click()

    title = driver.find_element_by_class_name("title")
    image_url = driver.find_element_by_link_text("Sample")

    hem_list.append({
        
        "title": title.text, 
        "img_url": image_url.get_attribute("href")
    })
    driver.back()

pprint(hem_list)
driver.close()

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]
